In [1]:
!pip install torch==2.0.1 transformers==4.30.2 datasets==2.13.0 trl==0.4.7 evaluate==0.4.0 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.1.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 17.0.0 which is incompatible.
kaggle-environments 1.16.9 requires transformers>=4.33.1, but you have transformers 4.30.2 which is incompatible.
pathos 0.3.3 requires dill>=0.3.9, but you have dill 0.3.6 which is incompatible.
pathos 0.3.3 requires multiprocess>=0.70.17, but you have multiprocess 0.70.14 which is incompatible.
pytorch-lightning 2.4.0 requires torch>=2.1.0, but you have torch 2.0.1 which is incompatible.


In [2]:

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead
from trl.core import LengthSampler
import numpy as np
import evaluate
from typing import List, Dict
from datasets import Dataset
import pandas as pd


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if device == "cpu":
    print("WARNING: No GPU detected. Training will be slow!")
elif torch.cuda.get_device_name():
    print(f"GPU detected: {torch.cuda.get_device_name()}")


Using device: cuda
GPU detected: Tesla P100-PCIE-16GB


In [4]:

# Python package versions:
# torch==2.0.1
# transformers==4.30.2
# datasets==2.13.0
# trl==0.4.7
# evaluate==0.4.0

def exact_match_score(prediction: str, ground_truth: str) -> float:
    """Calculate exact match score between prediction and ground truth."""
    return float(prediction.strip().lower() == ground_truth.strip().lower())

def tokenize_function(examples: Dict) -> Dict:
    """Tokenize the input examples."""
    questions = examples["question"]
    contexts = examples["context"]
    prompts = [f"Question: {q}\nContext: {c}\nAnswer:" for q, c in zip(questions, contexts)]
    return tokenizer(prompts, truncation=True, padding=True, max_length=512)

def get_rewards(predictions: List[str], ground_truths: List[str]) -> List[float]:
    """Calculate rewards for predictions using exact match score."""
    rewards = []
    for pred, truth in zip(predictions, ground_truths):
        reward = exact_match_score(pred, truth[0])  # taking first answer as ground truth
        rewards.append(reward)
    return rewards

# Initialize model and tokenizer
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(model).to(device)

splits = {'train': 'plain_text/train-00000-of-00001.parquet', 'validation': 'plain_text/validation-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/rajpurkar/squad/" + splits["train"])
df = df.sample(3000)
# Load and prepare dataset
dataset = Dataset.from_pandas(df)  # Using first 1000 examples for demonstration


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:

# Configure PPO training
config = PPOConfig(
    batch_size=16,
    learning_rate=5e-5,
    mini_batch_size=2,
)

# Initialize PPO trainer
ppo_trainer = PPOTrainer(
    config=config,
    model=model,
    tokenizer=tokenizer,
)


/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:235: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(


In [6]:
def get_rewards(predictions: List[str], ground_truths: List[List[str]]) -> List[float]:
    """Calculate rewards for predictions using exact match score."""
    rewards = []
    for pred, answers in zip(predictions, ground_truths):
        # Use any of the possible answers
        reward = max(exact_match_score(pred, ans) for ans in answers)
        rewards.append(reward)
    return rewards


In [7]:
print(len(dataset) // 20)

150


In [10]:
!pip install -U tqdm

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.4
    Uninstalling tqdm-4.66.4:
      Successfully uninstalled tqdm-4.66.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
beatrix-jupyterlab 2024.66.154055 requires jupyterlab~=3.6.0, but you have jupyterlab 4.3.1 which is incompatible.
conda 24.5.0 requires packaging>=23.0, but you have packaging 21.3 which is incom

In [ ]:
from tqdm import tqdm

# Training loop
total_batches = len(dataset) // 20
for epoch in tqdm(range(20), desc="Epochs"):
    epoch_rewards = []
    
    # Create progress bar for batches
    batch_iterator = tqdm(
        range(0, len(dataset), config.batch_size),
        desc=f"Epoch {epoch+1}/{20}",
        total=total_batches,
        leave=False
    )
    
    for batch_idx in batch_iterator:
        try:
                # Ensure we don't exceed dataset size
            batch_end_idx = min(batch_idx + config.batch_size, len(dataset))
            actual_batch_size = batch_end_idx - batch_idx
            batch = dataset[batch_idx:batch_end_idx]
            
            # Prepare query inputs
            questions = batch["question"]
            contexts = batch["context"]
            prompts = [f"Question: {q}\nContext: {c}\nAnswer:" for q, c in zip(questions, contexts)]
            
            # Process queries individually to create a list of tensors
            query_tensors = []
            response_tensors = []
            
            # Generate query tensors and responses in batch
            for prompt in prompts:
                # Tokenize query
                encoded = tokenizer(
                    prompt,
                    return_tensors="pt",
                    padding=True,
                    truncation=True,
                    max_length=512
                )
                query_tensor = encoded.input_ids.squeeze()
                query_tensors.append(query_tensor.to(device))
    
                # Generate response
                with torch.no_grad():
                    output = model.pretrained_model.generate(
                        input_ids=encoded.input_ids.to(device),
                        max_length=50,
                        do_sample=True,
                        temperature=1.0,
                        pad_token_id=tokenizer.pad_token_id,
                    )
                    response_tensors.append(output.squeeze().to(device))
            
            # Ensure all tensors have 1D shape
            query_tensors = [t.view(-1) for t in query_tensors]
            response_tensors = [t.view(-1) for t in response_tensors]
            
            # Decode responses
            batch_responses = [
                tokenizer.decode(response, skip_special_tokens=True)
                for response in response_tensors
            ]
            
            # Calculate rewards with proper answer handling
            answers = []
            for ans in batch["answers"]:
                # SQuAD format has multiple possible text answers per question
                answers.append(ans["text"])
                
            # Calculate rewards using the lists of possible answers
            rewards = get_rewards(batch_responses, answers)
            reward_tensors = [torch.tensor(reward, device=device) for reward in rewards]
            
     
            # Skip batch if sizes don't match
            if not (len(query_tensors) == len(response_tensors) == len(reward_tensors) == actual_batch_size):
                print(f"Skipping batch due to size mismatch: queries={len(query_tensors)}, responses={len(response_tensors)}, rewards={len(reward_tensors)}")
                continue
                
            # Run PPO step
            stats = ppo_trainer.step(query_tensors, response_tensors, reward_tensors)
            
            # Print training statistics
            mean_reward = torch.mean(torch.tensor([r.item() for r in reward_tensors]))
            # print(f"Epoch {epoch}, Batch {batch_idx}: Mean reward = {mean_reward:.3f}")
            
            # Update progress bar with reward info
            batch_iterator.set_postfix({
                'reward': f"{mean_reward:.3f}",
                'batch': f"{batch_idx//config.batch_size}/{total_batches}"
            })
            
            epoch_rewards.append(mean_reward)
        except: 
            print("Error at sample")
    # Print epoch summary
    avg_epoch_reward = sum(epoch_rewards) / len(epoch_rewards)
    print(f"\nEpoch {epoch+1} complete - Average reward: {avg_epoch_reward:.3f}")


Epoch 1/20:   0%|          | 0/150 [00:00<?, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1067: UserWarning: The average ratio of batch (28.59) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1067: UserWarning: The average ratio of batch (21.39) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1067: UserWarning: The average ratio of batch (14.00) exceeds threshold 10.00. Skipping batch.
  warnings.warn(

Epoch 1/20:  38%|███▊      | 57/150 [07:47<09:55,  6.41s/it, reward=0.688, batch=55/150]

Error at sample



Epoch 1/20: 100%|██████████| 150/150 [20:45<00:00,  9.02s/it, reward=0.688, batch=150/150]
Epoch 1/20: 151it [20:45,  8.72s/it, reward=0.688, batch=150/150]                         
Epoch 1/20: 151it [20:53,  8.72s/it, reward=0.438, batch=151/150]
Epoch 1/20: 152it [20:53,  8.62s/it, reward=0.438, batch=151/150]
Epoch 1/20: 152it [21:01,  8.62s/it, reward=0.500, batch=152/150]
Epoch 1/20: 153it [21:01,  8.28s/it, reward=0.500, batch=152/150]
Epoch 1/20: 153it [21:09,  8.28s/it, reward=0.625, batch=153/150]
Epoch 1/20: 154it [21:09,  8.11s/it, reward=0.625, batch=153/150]
Epoch 1/20: 154it [21:18,  8.11s/it, reward=0.562, batch=154/150]
Epoch 1/20: 155it [21:18,  8.38s/it, reward=0.562, batch=154/150]
Epoch 1/20: 155it [21:27,  8.38s/it, reward=0.625, batch=155/150]
Epoch 1/20: 156it [21:27,  8.79s/it, reward=0.625, batch=155/150]
Epoch 1/20: 156it [21:36,  8.79s/it, reward=0.438, batch=156/150]
Epoch 1/20: 157it [21:36,  8.86s/it, reward=0.438, batch=156/150]
Epoch 1/20: 157it [21:47,

Error at sample

Epoch 1 complete - Average reward: 0.520



Epoch 2/20:  22%|██▏       | 33/150 [04:38<16:25,  8.42s/it, reward=0.375, batch=32/150]/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1067: UserWarning: The average ratio of batch (40.50) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1067: UserWarning: The average ratio of batch (49.28) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1067: UserWarning: The average ratio of batch (43.62) exceeds threshold 10.00. Skipping batch.
  warnings.warn(

Epoch 2/20:  60%|██████    | 90/150 [12:30<08:24,  8.41s/it, reward=0.500, batch=89/150]/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1067: UserWarning: The average ratio of batch (14.55) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1067: UserWarning: The average ratio of

...

Error at sample



Epoch 14/20:  83%|████████▎ | 124/150 [14:14<01:55,  4.43s/it, reward=0.375, batch=122/150]

Error at sample



Epoch 14/20:  84%|████████▍ | 126/150 [14:23<01:42,  4.26s/it, reward=0.312, batch=124/150]

Error at sample


In [ ]:
ppo_trainer.save_pretrained("/kaggle/working/")